# Challenger: spaCy 
<img style="width: 20%; height: 20%;" src="https://usercontent1.hubstatic.com/14581778_f520.jpg">



In [118]:
# Use Natasha to generate the data. Thank you to Artjoms Šeļa for the suggestion to get data from Natasha.
import pandas as pd
from natasha import NamesExtractor

# For data, we're going to use a dataset of 6,700 works of poetry and short fiction from Zhurnal'nyi zal
# The full_journals_data.csv file can be downloaded from here: https://haverford.box.com/v/journals-data

data = pd.read_csv("/home/ajanco/Downloads/full_journals_data.csv")
data = data.text.tolist()

# problem names
import pandas as pd

df = pd.read_csv(
    "https://raw.githubusercontent.com/hingston/russian/master/10000-russian-words-cyrillic-only.txt",
    names=["name",],
    header=None,
)
problem_names = df.name.tolist()

Before we can use the output from Natasha as spaCy training data, we'll need to break Natasha's matches into individual token ids and spans. 

In [119]:
text = """Начиная жизнеописание героя моего, Алексея Федоровича Карамазова, нахожусь в некотором недоумении. А именно: хотя я и называю Алексея Федоровича моим героем, но однако сам знаю, что человек он отнюдь не великий, а посему и предвижу неизбежные вопросы в роде таковых: чем же замечателен ваш Алексей Федорович, что вы выбрали его своим героем? Что сделал он такого? Кому и чем известен? Почему я, читатель, должен тратить время на изучение фактов его жизни?
   Последний вопрос самый роковой, ибо на него могу лишь ответить: "Может быть увидите сами из романа". Ну а коль прочтут роман и не увидят, не согласятся с примечательностью моего Алексея Федоровича? Говорю так, потому что с прискорбием это предвижу. Для меня он примечателен, но решительно сомневаюсь, успею ли это доказать читателю. Дело в том, что это пожалуй и деятель, но деятель неопределенный, не выяснившийся. Впрочем странно бы требовать в такое время как наше от людей ясности. Одно, пожалуй, довольно несомненно: это человек странный, даже чудак. Но странность и чудачество скорее вредят, чем дают право на внимание, особенно когда все стремятся к тому, чтоб объединить частности и найти хоть какой-нибудь общий толк во всеобщей бестолочи. Чудак же в большинстве случаев частность и обособление. Не так ли?
   Вот если вы не согласитесь с этим последним тезисом, и ответите: "Не так" или "не всегда так", то я пожалуй и ободрюсь духом на счет значения героя моего Алексея Федоровича. Ибо не только чудак "не всегда" частность и обособление, а напротив бывает так, что он-то пожалуй и носит в себе иной раз сердцевину целого, а остальные люди его эпохи -- все, каким-нибудь наплывным ветром, на время почему-то от него оторвались..."""

extractor = NamesExtractor()
matches = extractor(text)
for match in matches:
    for token in match.tokens:

        if "Name" in token.forms[0].grams.values:
            print(token.value, token.span[0], token.span[1], "IMIA")
        if "Surn" in token.forms[0].grams.values:
            print(token.value, token.span[0], token.span[1], "FAMILIIA")
        if "Patr" in token.forms[0].grams.values:
            print(token.value, token.span[0], token.span[1], "OTCHESTVO")

Алексея 35 42 IMIA
Федоровича 43 53 OTCHESTVO
Карамазова 54 64 FAMILIIA
Алексея 126 133 IMIA
Федоровича 134 144 OTCHESTVO
Алексей 290 297 IMIA
Федорович 298 307 OTCHESTVO
Алексея 637 644 IMIA
Федоровича 645 655 OTCHESTVO
Алексея 1432 1439 IMIA
Федоровича 1440 1450 OTCHESTVO


Now that we're getting the data that we need, we can write it to the format that spaCy expects. 
Full documentation on training data can be found [here](https://spacy.io/usage/training#training-data).

- We're going to create a tuple with the text at index one and a dictionary containing a list of entitites at index two.  Each entity is a tuple with start character and end character in the text of the entity and the entity label.   

 ```('text', {'entities': [(52, 65, 'FAMILIIA'), (94, 102, 'FAMILIIA'),]})```
                                                      

In [121]:
import pickle
import random
from tqdm import tqdm

TRAINING_DATA = []

data = random.sample(data, 2000)  # A random sample of 2,000 of the total 6,700 records.

for text in tqdm(data):

    extractor = NamesExtractor()
    matches = extractor(text)
    entry = (text, {"entities": []})
    for match in matches:
        for token in match.tokens:
            if token.value not in problem_names:
                try:
                    if "Name" in token.forms[0].grams.values:
                        entry[1]["entities"].append(
                            (token.span[0], token.span[1], "IMIA")
                        )
                except Exception as e:
                    continue
                try:
                    if "Surn" in token.forms[0].grams.values:
                        entry[1]["entities"].append(
                            (token.span[0], token.span[1], "FAMILIIA")
                        )
                except Exception as e:
                    continue

                try:
                    if "Patr" in token.forms[0].grams.values:
                        entry[1]["entities"].append(
                            (token.span[0], token.span[1], "OTCHESTVO")
                        )
                except Exception as e:
                    continue
    TRAINING_DATA.append(entry)

pickle.dump(TRAINING_DATA, open("FROM_NATASHA_DATA.pickle", "wb"))





  0%|          | 0/2000 [00:00<?, ?it/s]



  0%|          | 1/2000 [00:01<38:12,  1.15s/it]



  0%|          | 2/2000 [00:01<30:08,  1.10it/s]



  0%|          | 3/2000 [00:04<48:51,  1.47s/it]



  0%|          | 4/2000 [00:05<43:08,  1.30s/it]



  0%|          | 6/2000 [00:09<51:57,  1.56s/it]



  0%|          | 7/2000 [00:09<39:44,  1.20s/it]



  0%|          | 8/2000 [00:10<29:45,  1.12it/s]



  0%|          | 10/2000 [00:13<35:27,  1.07s/it]



  1%|          | 12/2000 [00:14<32:50,  1.01it/s]



  1%|          | 13/2000 [00:14<25:59,  1.27it/s]



  1%|          | 14/2000 [00:15<22:10,  1.49it/s]



  1%|          | 15/2000 [00:16<26:36,  1.24it/s]



  1%|          | 16/2000 [00:16<20:30,  1.61it/s]



  1%|          | 18/2000 [00:19<28:18,  1.17it/s]



  1%|          | 19/2000 [00:21<39:28,  1.20s/it]



  1%|          | 20/2000 [00:24<58:39,  1.78s/it]



  1%|          | 21/2000 [00:28<1:15:12,  2.28s/it]



  1%|          | 22/2000 [00:28<1:01:31,  1.87s/it]



 

 16%|█▌        | 320/2000 [07:36<2:05:19,  4.48s/it]



 16%|█▌        | 322/2000 [07:37<1:29:45,  3.21s/it]



 16%|█▌        | 323/2000 [07:37<1:04:57,  2.32s/it]



 16%|█▌        | 324/2000 [07:37<46:39,  1.67s/it]  



 16%|█▋        | 325/2000 [07:39<47:03,  1.69s/it]



 16%|█▋        | 326/2000 [07:42<57:56,  2.08s/it]



 16%|█▋        | 327/2000 [07:43<50:10,  1.80s/it]



 16%|█▋        | 328/2000 [07:45<50:11,  1.80s/it]



 16%|█▋        | 329/2000 [07:46<43:53,  1.58s/it]



 16%|█▋        | 330/2000 [07:46<36:23,  1.31s/it]



 17%|█▋        | 332/2000 [07:47<28:27,  1.02s/it]



 17%|█▋        | 333/2000 [07:49<32:32,  1.17s/it]



 17%|█▋        | 334/2000 [07:49<27:18,  1.02it/s]



 17%|█▋        | 335/2000 [07:52<45:41,  1.65s/it]



 17%|█▋        | 336/2000 [07:56<1:02:24,  2.25s/it]



 17%|█▋        | 337/2000 [08:03<1:41:57,  3.68s/it]



 17%|█▋        | 338/2000 [08:03<1:13:21,  2.65s/it]



 17%|█▋        | 339/2000 [08:05<1:05:27,  2.36s/it]



 17%|█▋     

 32%|███▏      | 642/2000 [15:35<1:21:09,  3.59s/it]



 32%|███▏      | 643/2000 [15:37<1:11:14,  3.15s/it]



 32%|███▏      | 647/2000 [15:40<54:19,  2.41s/it]  



 32%|███▏      | 648/2000 [15:40<40:16,  1.79s/it]



 32%|███▏      | 649/2000 [15:43<47:04,  2.09s/it]



 32%|███▎      | 650/2000 [15:43<37:30,  1.67s/it]



 33%|███▎      | 651/2000 [15:44<27:36,  1.23s/it]



 33%|███▎      | 652/2000 [15:50<1:02:49,  2.80s/it]



 33%|███▎      | 653/2000 [15:50<45:39,  2.03s/it]  



 33%|███▎      | 654/2000 [15:50<32:42,  1.46s/it]



 33%|███▎      | 655/2000 [15:51<24:27,  1.09s/it]



 33%|███▎      | 657/2000 [15:52<21:35,  1.04it/s]



 33%|███▎      | 658/2000 [15:53<21:01,  1.06it/s]



 33%|███▎      | 659/2000 [15:53<16:19,  1.37it/s]



 33%|███▎      | 660/2000 [15:57<41:02,  1.84s/it]



 33%|███▎      | 661/2000 [15:59<40:28,  1.81s/it]



 33%|███▎      | 662/2000 [16:00<33:54,  1.52s/it]



 33%|███▎      | 663/2000 [16:02<39:14,  1.76s/it]



 33%|███▎      | 6

 49%|████▉     | 975/2000 [21:54<35:25,  2.07s/it]



 49%|████▉     | 976/2000 [21:57<37:44,  2.21s/it]



 49%|████▉     | 977/2000 [21:57<29:37,  1.74s/it]



 49%|████▉     | 978/2000 [22:05<1:00:52,  3.57s/it]



 49%|████▉     | 980/2000 [22:06<44:41,  2.63s/it]  



 49%|████▉     | 981/2000 [22:07<37:26,  2.20s/it]



 49%|████▉     | 982/2000 [22:10<42:36,  2.51s/it]



 49%|████▉     | 983/2000 [22:14<47:35,  2.81s/it]



 49%|████▉     | 984/2000 [22:14<34:52,  2.06s/it]



 49%|████▉     | 985/2000 [22:18<43:55,  2.60s/it]



 49%|████▉     | 987/2000 [22:20<35:05,  2.08s/it]



 49%|████▉     | 988/2000 [22:21<32:34,  1.93s/it]



 49%|████▉     | 989/2000 [22:23<28:51,  1.71s/it]



 50%|████▉     | 991/2000 [22:23<21:21,  1.27s/it]



 50%|████▉     | 993/2000 [22:28<26:11,  1.56s/it]



 50%|████▉     | 994/2000 [22:28<20:09,  1.20s/it]



 50%|████▉     | 997/2000 [22:32<20:13,  1.21s/it]



 50%|████▉     | 998/2000 [22:47<1:31:34,  5.48s/it]



 50%|████▉     | 999/2

 64%|██████▍   | 1288/2000 [29:52<22:21,  1.88s/it]



 64%|██████▍   | 1289/2000 [30:01<49:26,  4.17s/it]



 64%|██████▍   | 1290/2000 [30:02<35:49,  3.03s/it]



 65%|██████▍   | 1291/2000 [30:02<27:23,  2.32s/it]



 65%|██████▍   | 1292/2000 [30:03<21:03,  1.78s/it]



 65%|██████▍   | 1293/2000 [30:04<19:55,  1.69s/it]



 65%|██████▍   | 1297/2000 [30:07<15:58,  1.36s/it]



 65%|██████▍   | 1298/2000 [30:07<13:16,  1.13s/it]



 65%|██████▍   | 1299/2000 [30:09<13:54,  1.19s/it]



 65%|██████▌   | 1301/2000 [30:09<11:01,  1.06it/s]



 65%|██████▌   | 1302/2000 [30:11<12:45,  1.10s/it]



 65%|██████▌   | 1304/2000 [30:13<12:33,  1.08s/it]



 65%|██████▌   | 1305/2000 [30:14<13:20,  1.15s/it]



 65%|██████▌   | 1308/2000 [30:20<15:27,  1.34s/it]



 65%|██████▌   | 1309/2000 [30:20<13:15,  1.15s/it]



 66%|██████▌   | 1310/2000 [30:22<15:06,  1.31s/it]



 66%|██████▌   | 1311/2000 [30:25<22:02,  1.92s/it]



 66%|██████▌   | 1312/2000 [30:26<16:22,  1.43s/it]



 66%|█████

 81%|████████  | 1618/2000 [37:12<02:42,  2.35it/s]



 81%|████████  | 1619/2000 [37:12<02:30,  2.53it/s]



 81%|████████  | 1621/2000 [37:14<03:27,  1.82it/s]



 81%|████████  | 1622/2000 [37:14<03:21,  1.88it/s]



 81%|████████  | 1623/2000 [37:17<06:22,  1.01s/it]



 81%|████████  | 1624/2000 [37:17<06:02,  1.04it/s]



 81%|████████▏ | 1625/2000 [37:19<07:53,  1.26s/it]



 81%|████████▏ | 1626/2000 [37:25<15:27,  2.48s/it]



 81%|████████▏ | 1627/2000 [37:25<11:09,  1.79s/it]



 81%|████████▏ | 1628/2000 [37:26<10:22,  1.67s/it]



 81%|████████▏ | 1629/2000 [37:27<08:19,  1.35s/it]



 82%|████████▏ | 1630/2000 [37:39<27:41,  4.49s/it]



 82%|████████▏ | 1631/2000 [37:39<20:05,  3.27s/it]



 82%|████████▏ | 1634/2000 [37:39<14:10,  2.32s/it]



 82%|████████▏ | 1635/2000 [37:41<12:41,  2.09s/it]



 82%|████████▏ | 1636/2000 [37:43<12:24,  2.04s/it]



 82%|████████▏ | 1637/2000 [37:44<11:25,  1.89s/it]



 82%|████████▏ | 1638/2000 [37:45<08:24,  1.39s/it]



 82%|█████

 97%|█████████▋| 1942/2000 [45:16<01:04,  1.11s/it]



 97%|█████████▋| 1944/2000 [45:23<01:40,  1.79s/it]



 97%|█████████▋| 1945/2000 [45:26<01:56,  2.11s/it]



 97%|█████████▋| 1946/2000 [45:29<02:08,  2.38s/it]



 97%|█████████▋| 1947/2000 [45:31<02:06,  2.39s/it]



 97%|█████████▋| 1948/2000 [45:32<01:45,  2.03s/it]



 98%|█████████▊| 1950/2000 [45:34<01:24,  1.69s/it]



 98%|█████████▊| 1951/2000 [45:34<01:00,  1.23s/it]



 98%|█████████▊| 1952/2000 [45:35<00:52,  1.10s/it]



 98%|█████████▊| 1953/2000 [45:38<01:13,  1.57s/it]



 98%|█████████▊| 1955/2000 [45:38<00:53,  1.18s/it]



 98%|█████████▊| 1956/2000 [45:39<00:40,  1.08it/s]



 98%|█████████▊| 1957/2000 [45:39<00:29,  1.46it/s]



 98%|█████████▊| 1958/2000 [45:39<00:26,  1.56it/s]



 98%|█████████▊| 1959/2000 [45:40<00:29,  1.39it/s]



 98%|█████████▊| 1960/2000 [45:41<00:33,  1.20it/s]



 98%|█████████▊| 1961/2000 [45:42<00:25,  1.55it/s]



 98%|█████████▊| 1962/2000 [45:42<00:20,  1.86it/s]



 98%|█████

In [122]:
# Inspect the results.  This is an essential step, because this text is the standard from which the machine will learn.
# I spent several days wondering why the results were so poor, turns out, bad data in - bad model out.

from spacy.lang.ru import Russian
from spacy import displacy
from spacy.pipeline import EntityRuler


nlp = Russian()
ruler = EntityRuler(nlp)
patterns = []
for text, annotations in TRAINING_DATA[:1]:

    # for text, annotations in TRAINING_DATA:
    ents = annotations["entities"]
    ents
    # ruler.add_patterns([{"label": "ORG", "pattern": "Apple"}])
    for ent in ents:
        patterns.append({"label": ent[2], "pattern": text[ent[0] : ent[1]]})
    ruler.add_patterns(patterns)
    nlp.add_pipe(ruler)
    doc = nlp(text)

    displacy.render(doc, style="ent")

Now for the best part - learning! In the following cell, I split the data into training and test sets.  This is important, because we want to evaluate how well the model works with texts that it has never seen before.  To keep things simple, we work from spaCys base Russian object. We add a named entity recognition pipeline to the object and add the labels for our three categories.

We then begin training! We split the data into smaller minibatches and update the model on each batch.  Once all the minibatches have been served to the model, we have completed one iteration.  

In [123]:
import random
import pickle
from tqdm import tqdm

from sklearn.model_selection import train_test_split

# Here were create test data for the final evaluation.
train, test = train_test_split(TRAINING_DATA, test_size=0.2)
len(train)


import spacy

nlp = Russian()


# Start the training
if not nlp.has_pipe("ner"):
    nlp.add_pipe(nlp.create_pipe('ner'))
ner = nlp.get_pipe("ner")
ner.add_label("IMIA")
ner.add_label("OTCHESTVO")
ner.add_label('FAMILIIA')

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
with nlp.disable_pipes(*other_pipes):  # only train NER

    nlp.begin_training()
    # Loop for 10 iterations
    for itn in tqdm(range(10)):
        # Shuffle the training data
        random.shuffle(train)
        losses = {}
        
        # Batch the examples and iterate over them
        for batch in spacy.util.minibatch(train, size=2):
            texts = [text for text, entities in batch]
            annotations = [entities for text, entities in batch]

            # Update the model
            try:
                nlp.update(texts, annotations, losses=losses)
            except ValueError:
                continue
            #print(losses)
    nlp.to_disk("./ru_names_from_natasha")
pickle.dump(test, open( "TEST.pickle", "wb" ) )





  0%|          | 0/10 [00:00<?, ?it/s]



 10%|█         | 1/10 [09:22<1:24:23, 562.61s/it]



 20%|██        | 2/10 [18:04<1:13:22, 550.36s/it]



 30%|███       | 3/10 [26:45<1:03:10, 541.55s/it]



 40%|████      | 4/10 [35:27<53:34, 535.81s/it]  



 50%|█████     | 5/10 [44:27<44:45, 537.06s/it]



 60%|██████    | 6/10 [53:18<35:40, 535.02s/it]



 70%|███████   | 7/10 [1:02:09<26:41, 533.89s/it]



 80%|████████  | 8/10 [1:10:54<17:42, 531.35s/it]



 90%|█████████ | 9/10 [1:23:12<09:53, 593.15s/it]



100%|██████████| 10/10 [1:32:30<00:00, 555.08s/it]


# Now let's evaluate the model.
```
UAS (Unlabelled Attachment Score) and LAS (Labelled Attachment Score) are standard metrics to evaluate dependency parsing. UAS is the proportion of tokens whose head has been correctly assigned, LAS is the proportion of tokens whose head has been correctly assigned with the right dependency label (subject, object, etc).
ents_p, ents_r, ents_f are the precision, recall and fscore for the NER task.
tags_acc is the POS tagging accuracy.
token_acc seems to be the precision for token segmentation.```([source](https://stackoverflow.com/questions/50644777/understanding-spacys-scorer-output))

In [151]:
test = pickle.load(open( "TEST.pickle", "rb" ) )
test = [(text, annotations['entities']) for text, annotations in test]

import spacy
from spacy.scorer import Scorer

def evaluate(ner_model, examples):
    scorer = Scorer()
    for input_, annot in examples:
        doc_gold_text = ner_model.make_doc(input_)
        gold = GoldParse(doc_gold_text, entities=annot)
        pred_value = ner_model(input_)
        scorer.score(pred_value, gold)
    return scorer.scores

evaluate(spacy.load("./ru_names_from_natasha"), test)

{'uas': 0.0,
 'las': 0.0,
 'ents_p': 85.13628218428747,
 'ents_r': 83.61430159318266,
 'ents_f': 84.3684284312351,
 'ents_per_type': {'IMIA': {'p': 87.984555984556,
   'r': 80.64835787089469,
   'f': 84.15688012408597},
  'FAMILIIA': {'p': 76.03179455823907,
   'r': 87.35511064278188,
   'f': 81.30107878391631},
  'OTCHESTVO': {'p': 98.36639439906651,
   'r': 95.25423728813558,
   'f': 96.78530424799082}},
 'tags_acc': 0.0,
 'token_acc': 100.0}

In [124]:
# The model with 2000 training texts and 10 iterations

import spacy
from spacy import displacy 
nlp = spacy.load("./ru_names_from_natasha")

bk = open('bk.txt','r').read()
doc = nlp(bk)
displacy.render(doc, style="ent")

# Challenger: Prodigy 
<img style="width: 20%; height: 20%;" src="http://images5.fanpop.com/image/photos/27100000/Suzzana-s-Pokemon-mariposa-region-rpg-27148273-2350-1933.png">

- Prodigy is an active learning tool from the makers of spaCy.  
- I began by manually annotating texts with imiia otchestvo and familiia. 
- I then used Prodigy to add annotations for ambiguous results, thus strategically improving the model's predictions.  
- The model was then trained on the annotations.  


`$ prodigy dataset ru_names_from_natasha`


✨  Successfully added 'ru_names_from_natasha' to database SQLite.

`$ prodigy db-in ru_names_from_natasha ./bk_zhivago.jsonl/luna-cat.jsonl` 

✨  Imported 100 annotations for 'ru_names_from_natasha' to database SQLite
  Added 'accept' answer to 0 annotations
  Session ID: 2019-11-19_08-16-16

`$ prodigy ner.teach ru_names_from_natasha ./ru_names_from_natasha bk.txt --label IMIA,OTCHESTVO,FAMILIIA`

Using 3 labels: IMIA, OTCHESTVO, FAMILIIA

  ✨  Starting the web server at http://localhost:8080 ...
  Open the app in your browser and start annotating!

^C

Saved 128 annotations to database SQLite
Dataset: ru_names_from_natasha
Session ID: 2019-11-19_08-20-44


`$ prodigy ner.teach ru_names_from_natasha ./ru_names_from_natasha zhivago.txt --label IMIA,OTCHESTVO,FAMILIIA`

Using 3 labels: IMIA, OTCHESTVO, FAMILIIA

  ✨  Starting the web server at http://localhost:8080 ...
  Open the app in your browser and start annotating!

Task queue depth is 1
Task queue depth is 2
^C

Saved 43 annotations to database SQLite
Dataset: ru_names_from_natasha
Session ID: 2019-11-19_08-25-10

`$ prodigy ner.batch-train ru_names_from_natasha ./ru_names_from_natasha --label IMIA,OTCHESTVO,FAMILIIA --output ezhik --n-iter 10`

```text
Using 3 labels: IMIA, OTCHESTVO, FAMILIIA

Loaded model ./ru_names_from_natasha
Using 50% of accept/reject examples (62) for evaluation
Using 100% of remaining examples (176) for training
Dropout: 0.2  Batch size: 4  Iterations: 10  


BEFORE      0.826            
Correct     133
Incorrect   28
Entities    185              
Unknown     52               

#            LOSS         RIGHT        WRONG        ENTS         SKIP         ACCURACY  
01           44.814       150          11           203          0            0.932     
02           14.034       153          8            206          0            0.950     
03           2.778        155          6            209          0            0.963     
04           5.403        152          9            207          0            0.944     
05           1.915        152          9            206          0            0.944     
06           2.806        150          11           203          0            0.932     
07           0.115        152          9            210          0            0.944     
08           0.004        151          10           209          0            0.938     
09           0.012        151          10           209          0            0.938     
10           0.004        151          10           210          0            0.938     

Correct     155
Incorrect   6 
Baseline    0.826            
Accuracy    0.963            


Model: /home/ajanco/projects/RussianNLP/ezhik
Training data: /home/ajanco/projects/RussianNLP/ezhik/training.jsonl
Evaluation data: /home/ajanco/projects/RussianNLP/ezhik/evaluation.jsonl
```

In [150]:
# Download ezhik from here: https://haverford.box.com/v/ezhik-imiia-otchestva-familiia
# Then pip install ru_ezhik-0.0.0.tar.gz

test = pickle.load(open( "TEST.pickle", "rb" ) )
test = [(text, annotations['entities']) for text, annotations in test]

import spacy
from spacy.scorer import Scorer

def evaluate(ner_model, examples):
    scorer = Scorer()
    for input_, annot in examples:
        doc_gold_text = ner_model.make_doc(input_)
        gold = GoldParse(doc_gold_text, entities=annot)
        pred_value = ner_model(input_)
        scorer.score(pred_value, gold)
    return scorer.scores

evaluate(spacy.load('ru_ezhik'), test)

{'uas': 0.0,
 'las': 0.0,
 'ents_p': 76.50390457392946,
 'ents_r': 82.57688032604669,
 'ents_f': 79.42447325047887,
 'ents_per_type': {'IMIA': {'p': 79.17540181691125,
   'r': 80.19535673839184,
   'f': 79.68211547928827},
  'FAMILIIA': {'p': 68.06192660550458,
   'r': 83.38602037232175,
   'f': 74.94869771112866},
  'OTCHESTVO': {'p': 86.73267326732673,
   'r': 98.98305084745763,
   'f': 92.45382585751979}},
 'tags_acc': 0.0,
 'token_acc': 100.0}

In [134]:
# The model with fine tuning on 171 annotations and 10 iterations with Prodigy

import spacy
from spacy import displacy 
nlp = spacy.load('ru_ezhik')

bk = open('bk.txt','r').read()
doc = nlp(bk)
displacy.render(doc, style="ent")

# Who won? demonstrate that Ezhik can identify oov terms where Natasha cannot. 


In [172]:
text = """ДЕЙСТВУЮТ:


     1. Чтец.
     2. Графиня Елена Васильевна Безухова (Элен).
     3. Граф Петр Кириллович Безухое (Пьер).
     4. Князь Анатолий Васильевич Курагин (Анатоль).
     5. Княжна Марья Николаевна Волконская (Марья).
     6. Князь Андрей Николаевич Болконский (Андрей).
     7. Князь Николай Андреевич Болконский (Болконский).
     8. Графиня Наталья Ильинична Ростова (Наташа).
     9. Графиня Ростова - мать (Графиня)
     10.Граф Ростов - отец (Граф).
     11. Граф Петр Ильич Ростов (Петя).
     12. Граф Николай Ильич Ростов (Ростов).
     13. Соня, племянница графа Ростова (Соня).
     14. Император Александр I (Александр).
     15. Растопчин.
     16. Кутузов, светлейший князь.
     17. Моряк-либерал.
     18. Сенатор.
     19. Апраксин, Степан Степанович.
     20. Нехороший игрок.
     21. Глинка, писатель.
     22. Шиншин, московский остряк.
     23. Ильин, гусарский офицер.
     24. Принц Виртембергский.
     25. Щербинин.
     26. Ермолов.
     27. Вольцоген, флигель-адъютант.
     28. Раевский.
     29. Кайсаров.
     30. Адъютант Кутузова.
     31. Другой адъютант.
     32. Еще адъютант.
     33. Неизвестный адъютант.
     34. Доктор.
     35. Бледный офицер.
     36. Майор.
     37. Макар Алексеевич.
     38. Человечек в вицмундире.
     39. Марья Николаевна, потерявшая ребенка.
     40. Красавица-армянка.
     41. Старик.
     42. Толь.
     43. Болховитинов.
     44. Генерал.
     45. Денисов.
     46. Долохов.
     47. Эсаул.
     48. Ливрейный лакей Ростовых.
     49. Лаврушка, денщик Николая Ростова.
     50. Тихон, камердинер Болконского.
     51. Алпатыч.
     52. Дуняша, горничная Волконской.
     53. Дрон, староста.
     54. Длинный мужик.
     55. Один мужик.
     56. Небольшой мужик.
     57. Карп.
     58. Круглолицый мужик.
     59. Повар Кутузова.
     60. Денщик Кутузова.
     61. Черноволосый унтер-офицер.
     62. Раненый солдат.
     63. Фельдшер 1-й.
     64. Фельдшер 2-й.
     65. Солдат с котелком.
     66. Берейтор.
     67. Мавра Кузьминишна, ключница Ростовых.
     68. Васильич, дворецкий Ростовых.
     69. Буфетчик Ростовых.
     70. Слуга Ростовых.
     71. Почтенный камердинер Андрея.
     72. Денщик бледного офицера.
     73. Матрена Тимофеевна, шеф жандармов у Ростовых.
     74. Горничная Ростовых.
     75. Герасим, камердинер Баздеева.
     76. Кухарка Баздеева.
     77. Рябая баба.
     78. Первый острожный.
     79. Второй острожный.
     80. Дворовый, лет 45.
     81. Очень красивый мужик.
     82. Желтый фабричный.
     83. Тихон, партизан.
     84. Пленный русский солдат.
     85. Каратаев.
     86. Краснорожий мушкетер.
     87. Востроносенький мушкетер.
     88. Молодой мушкетер.
     89. Плясун-мушкетер.
     90. Старый мушкетер.
     91. Фельдфебель I.
     92. Фельдфебель II.
     93. Песельник-мушкетер.
     94. Вышедший мушкетер.
     95. Откупщик.
     96. Голова.
     97. Наполеон.
     98. Паж Наполеона.
     99. Маршал Бертье.
     100. Лелорм-Дидевиль, переводчик.
     101. Адъютант Наполеона.
     102. Граф Рамбаль.
     103. Морель, денщик Рамбаля.
     104. Маленький мародер, француз.
     105. Мародер в капоте, француз.
     106. Французский улан.
     107. Французский улан-офицер.
     108. Маленький человечек, француз.
     109. Маршал Даву.
     110. Адъютант Даву.
     111. Первый французский синий солдат.
     112. Второй французский синий солдат.
     113. Босс, барабанщик, француз.
     114. Француз-конвоир.
     115. Официант у Болконских."""
    

In [173]:
import spacy

from spacy import displacy 
nlp = spacy.load('ru_ezhik')

doc = nlp(text)
displacy.render(doc, style="ent")

In [174]:
matches = extractor(text)
matches

Matches('ДЕЙСТВУЮТ:\n\n\n     1. Чтец.\n     2. Графиня Елена Васильевна Безухова (Элен).\n     3. Граф Петр Кириллович Безухое (Пьер).\n     4. Князь Анатолий Васильевич Курагин (Анатоль).\n     5. Княжна Марья Николаевна Волконская (Марья).\n     6. Князь Андрей Николаевич Болконский (Андрей).\n     7. Князь Николай Андреевич Болконский (Болконский).\n     8. Графиня Наталья Ильинична Ростова (Наташа).\n     9. Графиня Ростова - мать (Графиня)\n     10.Граф Ростов - отец (Граф).\n     11. Граф Петр Ильич Ростов (Петя).\n     12. Граф Николай Ильич Ростов (Ростов).\n     13. Соня, племянница графа Ростова (Соня).\n     14. Император Александр I (Александр).\n     15. Растопчин.\n     16. Кутузов, светлейший князь.\n     17. Моряк-либерал.\n     18. Сенатор.\n     19. Апраксин, Степан Степанович.\n     20. Нехороший игрок.\n     21. Глинка, писатель.\n     22. Шиншин, московский остряк.\n     23. Ильин, гусарский офицер.\n     24. Принц Виртембергский.\n     25. Щербинин.\n     26. Ермолов.\n     27. Вольцоген, флигель-адъютант.\n     28. Раевский.\n     29. Кайсаров.\n     30. Адъютант Кутузова.\n     31. Другой адъютант.\n     32. Еще адъютант.\n     33. Неизвестный адъютант.\n     34. Доктор.\n     35. Бледный офицер.\n     36. Майор.\n     37. Макар Алексеевич.\n     38. Человечек в вицмундире.\n     39. Марья Николаевна, потерявшая ребенка.\n     40. Красавица-армянка.\n     41. Старик.\n     42. Толь.\n     43. Болховитинов.\n     44. Генерал.\n     45. Денисов.\n     46. Долохов.\n     47. Эсаул.\n     48. Ливрейный лакей Ростовых.\n     49. Лаврушка, денщик Николая Ростова.\n     50. Тихон, камердинер Болконского.\n     51. Алпатыч.\n     52. Дуняша, горничная Волконской.\n     53. Дрон, староста.\n     54. Длинный мужик.\n     55. Один мужик.\n     56. Небольшой мужик.\n     57. Карп.\n     58. Круглолицый мужик.\n     59. Повар Кутузова.\n     60. Денщик Кутузова.\n     61. Черноволосый унтер-офицер.\n     62. Раненый солдат.\n     63. Фельдшер 1-й.\n     64. Фельдшер 2-й.\n     65. Солдат с котелком.\n     66. Берейтор.\n     67. Мавра Кузьминишна, ключница Ростовых.\n     68. Васильич, дворецкий Ростовых.\n     69. Буфетчик Ростовых.\n     70. Слуга Ростовых.\n     71. Почтенный камердинер Андрея.\n     72. Денщик бледного офицера.\n     73. Матрена Тимофеевна, шеф жандармов у Ростовых.\n     74. Горничная Ростовых.\n     75. Герасим, камердинер Баздеева.\n     76. Кухарка Баздеева.\n     77. Рябая баба.\n     78. Первый острожный.\n     79. Второй острожный.\n     80. Дворовый, лет 45.\n     81. Очень красивый мужик.\n     82. Желтый фабричный.\n     83. Тихон, партизан.\n     84. Пленный русский солдат.\n     85. Каратаев.\n     86. Краснорожий мушкетер.\n     87. Востроносенький мушкетер.\n     88. Молодой мушкетер.\n     89. Плясун-мушкетер.\n     90. Старый мушкетер.\n     91. Фельдфебель I.\n     92. Фельдфебель II.\n     93. Песельник-мушкетер.\n     94. Вышедший мушкетер.\n     95. Откупщик.\n     96. Голова.\n     97. Наполеон.\n     98. Паж Наполеона.\n     99. Маршал Бертье.\n     100. Лелорм-Дидевиль, переводчик.\n     101. Адъютант Наполеона.\n     102. Граф Рамбаль.\n     103. Морель, денщик Рамбаля.\n     104. Маленький мародер, француз.\n     105. Мародер в капоте, француз.\n     106. Французский улан.\n     107. Французский улан-офицер.\n     108. Маленький человечек, француз.\n     109. Маршал Даву.\n     110. Адъютант Даву.\n     111. Первый французский синий солдат.\n     112. Второй французский синий солдат.\n     113. Босс, барабанщик, француз.\n     114. Француз-конвоир.\n     115. Официант у Болконских.',
        [Match([MorphTagToken('Елена',
                              [43, 48),
                              'RU',
                              'I',
                              [Form('елена',
                                    Grams(NOUN,Name,anim,femn,nomn,sing))]),
                MorphTagToken('Васильевна',
                              [49, 59),
                              'RU',
 

# #TODO Package the model in a way that is simple and useful. 
- Need to cluster names to identify distinct persons
- Return a list of distinct persons with a list of their appearances in the text 
-

In [136]:
# matcher
from spacy.matcher import Matcher

matcher = Matcher(nlp.vocab)
pattern = [{"ENT_TYPE": "IMIA"}]
matcher.add("IMIA", None, pattern)

pattern = [{"ENT_TYPE": "IMIA"}, {"ENT_TYPE": "OTCHESTVO"}]
matcher.add("IMIA-OTCHESTVO", None, pattern)

pattern = [{"ENT_TYPE": "IMIA"}, {"ENT_TYPE": "OTCHESTVO"}, {"ENT_TYPE": "FAMILIIA"}]
matcher.add("IMIA-OTCHESTVO-FAMILIIA", None, pattern)

pattern = [{"ENT_TYPE": "FAMILIIA"}, {"ENT_TYPE": "IMIA"}, {"ENT_TYPE": "OTCHESTVO", 'OP': '?'}]
matcher.add("FAMILIIA-IMIA-OTCHESTVO", None, pattern)

matches = matcher(doc)

for match_id, start, stop in matches:
    print(doc[start:stop], start,stop, doc.vocab.strings[match_id])

Федор 0 1 IMIA
Федор Михайлович 0 2 IMIA-OTCHESTVO
Федор Михайлович Достоевский 0 3 IMIA-OTCHESTVO-FAMILIIA
Анне 15 16 IMIA
Анне Григорьевне 15 17 IMIA-OTCHESTVO
Анне Григорьевне Достоевской 15 18 IMIA-OTCHESTVO-FAMILIIA
Иоанна 55 56 IMIA
Алексея 73 74 IMIA
Алексея Федоровича 73 75 IMIA-OTCHESTVO
Алексея Федоровича Карамазова 73 76 IMIA-OTCHESTVO-FAMILIIA
Алексея 89 90 IMIA
Алексея Федоровича 89 91 IMIA-OTCHESTVO
Алексей 120 121 IMIA
Алексей Федорович 120 122 IMIA-OTCHESTVO
Алексея 190 191 IMIA
Алексея Федоровича 190 192 IMIA-OTCHESTVO
Алексея 346 347 IMIA
Алексея Федоровича 346 348 IMIA-OTCHESTVO
ФЕДОР 834 835 IMIA
ФЕДОР ПАВЛОВИЧ 834 836 IMIA-OTCHESTVO
ФЕДОР ПАВЛОВИЧ КАРАМАЗОВ 834 837 IMIA-OTCHESTVO-FAMILIIA
Алексей 839 840 IMIA
Алексей Федорович 839 841 IMIA-OTCHESTVO
Алексей Федорович Карамазов 839 842 IMIA-OTCHESTVO-FAMILIIA
Федора 848 849 IMIA
Федора Павловича 848 850 IMIA-OTCHESTVO
Федора Павловича Карамазова 848 851 IMIA-OTCHESTVO-FAMILIIA
Федор 964 965 IMIA
Федор Павлович 964 9

Дмитрия Федоровича 44414 44416 IMIA-OTCHESTVO
Софью 44429 44430 IMIA
Софью Ивановну 44429 44431 IMIA-OTCHESTVO
Григорий 44498 44499 IMIA
Марфа 44552 44553 IMIA
Марфа Игнатьевна 44552 44554 IMIA-OTCHESTVO
Григорий 44629 44630 IMIA
Марфа 44641 44642 IMIA
Марфа Игнатьевна 44641 44643 IMIA-OTCHESTVO
Аделаиды 44696 44697 IMIA
Аделаиды Ивановны 44696 44698 IMIA-OTCHESTVO
Федором 44699 44700 IMIA
Федором Павловичем 44699 44701 IMIA-OTCHESTVO
Марфа 44732 44733 IMIA
Марфа Игнатьевна 44732 44734 IMIA-OTCHESTVO
Григорий 44786 44787 IMIA
Марфа 44831 44832 IMIA
Марфа Игнатьевна 44831 44833 IMIA-OTCHESTVO
Григорий 44855 44856 IMIA
Дмитрия 44873 44874 IMIA
Дмитрия Федоровича 44873 44875 IMIA-OTCHESTVO
Аделаида 44884 44885 IMIA
Аделаида Ивановна 44884 44886 IMIA-OTCHESTVO
Иваном 44912 44913 IMIA
Иваном Федоровичем 44912 44914 IMIA-OTCHESTVO
Алешей 44917 44918 IMIA
Марфа 44944 44945 IMIA
Марфа Игнатьевна 44944 44946 IMIA-OTCHESTVO
Увидя 44972 44973 IMIA
Григорий 44975 44976 IMIA
Григорий 45021 45022 IM

In [ ]:
#TODO Cluster by proximity and similarity of match 